In [1]:
! pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# =========================
# 0) Imports, engine check, folders
# =========================
import pandas as pd, numpy as np, warnings, re
from pathlib import Path
warnings.filterwarnings('ignore')

# --- Require a Parquet engine ---
try:
    import pyarrow  # noqa: F401
    PARQUET_ENGINE = "pyarrow"
except ImportError:
    try:
        import fastparquet  # noqa: F401
        PARQUET_ENGINE = "fastparquet"
    except ImportError as e:
        raise ImportError(
            "Parquet output is required but no engine is installed.\n"
            "Install one of:\n"
            "  pip install pyarrow\n"
            "    or\n"
            "  pip install fastparquet"
        ) from e

RAW  = Path('./raw')            # change if your files are elsewhere
PROC = Path('./processed'); PROC.mkdir(exist_ok=True)

In [3]:
# Robust column picker (case-insensitive, partial matches allowed)
def pick(df, candidates):
    for c in candidates:
        if c in df.columns: return c
    for c in df.columns:
        for k in candidates:
            if k.lower() in c.lower(): return c
    return None

# Smart date parser: chooses the best of multiple formats by coverage
from dateutil import parser as _dateparser

def smart_parse_dates(s: pd.Series) -> pd.Series:
    s = s.astype(str)
    attempts = []
    attempts.append(('infer', pd.to_datetime(s, errors='coerce')))
    attempts.append(('dayfirst', pd.to_datetime(s, errors='coerce', dayfirst=True)))
    fmts = ['%d-%b-%Y','%d-%b-%y','%d/%m/%Y','%m/%d/%Y','%Y-%m-%d','%b %d, %Y','%d %b %Y','%b %Y','%m-%Y']
    for fmt in fmts:
        attempts.append((fmt, pd.to_datetime(s, format=fmt, errors='coerce')))
    def score(dt):
        ok = dt.dropna()
        if ok.empty: return (0,0,0)
        ser = pd.Series(1, index=ok).sort_index()
        by_year_months = ser.groupby(ser.index.year).apply(lambda x: len(pd.Index(x.index.month).unique()))
        med_months = int(by_year_months.median()) if len(by_year_months) else 0
        return (len(ok), med_months, len(pd.Index(ok.dt.month).unique()))
    best_name, best_dt, best_score = max([(n,d,score(d)) for n,d in attempts], key=lambda t: t[2])
    print(f"[smart_parse_dates] picked: {best_name} | parsed={best_score[0]} | median months/yr={best_score[1]} | unique months={best_score[2]}")
    return best_dt

def parse_dates_in_df(df, date_col='Date'):
    out = df.copy()
    out[date_col] = smart_parse_dates(out[date_col])
    out = out.dropna(subset=[date_col]).sort_values(date_col)
    return out

def _parse_investing_prices(df, date_col='Date', price_col='Price'):
    out = parse_dates_in_df(df, date_col=date_col)
    out[price_col] = pd.to_numeric(out[price_col].astype(str).str.replace(',', '', regex=False), errors='coerce')
    out = out.dropna(subset=[price_col])
    return out[[date_col, price_col]]

def investing_to_quarter_ret(df, date_col='Date', price_col='Price'):
    df2 = _parse_investing_prices(df, date_col, price_col)
    q_end_price = df2.set_index(date_col)[price_col].resample('Q').last()
    return q_end_price.pct_change()

def monthly_to_quarter(series, how='mean'):
    return series.resample('Q').mean() if how=='mean' else series.resample('Q').last()

def yoy_from_monthly(series):
    return series.pct_change(12) * 100.0

def check_monthly_coverage(df, date_col='Date', label='series'):
    dates = smart_parse_dates(df[date_col]).dropna()
    ser = pd.Series(1, index=dates).sort_index()
    months_per_year = ser.groupby(ser.index.year).apply(lambda s: sorted(pd.Index(s.index.month).unique()))
    print(f"[{label}] Months present per year:")
    for y, months in months_per_year.items():
        print(f"  {y}: {months}")
    q_counts = ser.resample('Q').size()
    miss = q_counts[q_counts == 0]
    if len(miss) > 0:
        print(f"[{label}] ⚠ Missing {len(miss)} quarter(s) with zero rows — resample will yield NaNs.")
    else:
        print(f"[{label}] ✅ At least one row in every quarter.")


In [4]:
# Ensure files exist in ./raw
n50_raw = pd.read_csv(RAW/'Nifty50.csv')
mid_raw = pd.read_csv(RAW/'NIFTYMidcap100.csv')

# Quick coverage QA (helps catch wrong date parsing or partial exports)
check_monthly_coverage(n50_raw, 'Date', 'NIFTY 50')
check_monthly_coverage(mid_raw, 'Date', 'NIFTY Midcap 100')

nifty_qret  = investing_to_quarter_ret(n50_raw).rename('nifty_qret')
midcap_qret = investing_to_quarter_ret(mid_raw).rename('midcap_qret')
excess_ret  = (midcap_qret - nifty_qret).rename('excess_ret')

print("Index quarterly points (excess_ret):", excess_ret.dropna().shape[0])


[smart_parse_dates] picked: infer | parsed=188 | median months/yr=12 | unique months=12
[NIFTY 50] Months present per year:
  2010: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
  2011: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
  2012: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
  2013: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
  2014: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
  2015: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
  2016: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
  2017: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
  2018: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
  2019: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
  2020: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
  2021: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
  2022: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
  2023: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
  2024: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
  2025: [1, 2, 3, 4, 5, 6, 7, 8]
[NIFTY 50] ✅ At least one row in every quarter.
[smart_parse_dates] picked: infer | parsed=188 | median months/yr=12 | uniq

In [5]:
# CPI monthly to YoY% to quarterly mean
cpi = pd.read_csv(RAW/'CPI_Monthly_Jan_2013_to_Jun_2025.csv')  # <- file name you uploaded
c_date = pick(cpi, ['DATE','Date','Month','Period'])           # will choose 'DATE'
c_val  = pick(cpi, ['CPI','Index','Value','CPI_COMBINED_BASE2012_100'])  # will choose 'CPI_COMBINED_BASE2012_100'

cpi = parse_dates_in_df(cpi, c_date)   # smart date parser picked "infer" in my check
cpi_m = cpi.set_index(c_date)[c_val].astype(float).rename('cpi_index')

# YoY requires 12 months of history, so the first 12 rows will be NaN (expected)
cpi_yoy_m = yoy_from_monthly(cpi_m).rename('cpi_yoy_m')

# Aggregate to quarterly (mean of three months)
cpi_yoy_q = monthly_to_quarter(cpi_yoy_m, how='mean').rename('cpi_yoy')
print("CPI monthly points:", cpi_m.shape[0], "| CPI YoY monthly (non-NaN):", cpi_yoy_m.dropna().shape[0])
print("CPI quarterly points:", cpi_yoy_q.dropna().shape[0])


[smart_parse_dates] picked: infer | parsed=150 | median months/yr=12 | unique months=12
CPI monthly points: 150 | CPI YoY monthly (non-NaN): 138
CPI quarterly points: 46


In [6]:
# --- GDP quarterly loader (fixes '2012.0-Q1' etc.) ---

import re
from pandas.tseries.offsets import QuarterEnd

gdp_raw = pd.read_csv(RAW/'GDP_Quarterly_2010_2025.csv').copy()

# 1) Clean numeric column
if 'gdp' not in gdp_raw.columns:
    raise ValueError("Expected a 'gdp' column in GDP_Quarterly_2010_2025.csv")
gdp_raw['gdp_clean'] = pd.to_numeric(
    gdp_raw['gdp'].astype(str).str.replace(',', '', regex=False),
    errors='coerce'
)

# 2) Normalize quarter strings (strip the '.0' before '-Q')
if 'quarter' not in gdp_raw.columns:
    raise ValueError("Expected a 'quarter' column in GDP_Quarterly_2010_2025.csv")

gdp_raw['quarter_str'] = gdp_raw['quarter'].astype(str).str.replace('.0', '', regex=False)

# 3) Extract YYYY and Qn from anything like '2012-Q1', '2012Q1', 'Q1 2012.0', etc.
m = gdp_raw['quarter_str'].str.extract(r'(?P<y>\d{4}).*?Q(?P<q>[1-4])')
qstr = (m['y'].fillna('') + 'Q' + m['q'].fillna(''))
valid = qstr.str.match(r'^\d{4}Q[1-4]$')

gdp_norm = gdp_raw.loc[valid].copy()
gdp_norm['__q'] = pd.PeriodIndex(qstr[valid], freq='Q').to_timestamp(how='end')

# 4) Build a clean quarterly level series (last obs per quarter if duplicates)
gdp_q_level = (gdp_norm
               .dropna(subset=['__q','gdp_clean'])
               .set_index('__q')
               .sort_index()
               .groupby(pd.Grouper(freq='Q'))['gdp_clean']
               .last())

# 5) Compute YoY from levels (4-quarter change * 100)
gdp_q = gdp_q_level.pct_change(4).mul(100.0).rename('gdp_yoy').dropna()

# 6) Diagnostics
print("Unique 'quarter' samples:", gdp_raw['quarter'].astype(str).unique()[:8])
print("Normalized samples:", gdp_norm['quarter_str'].astype(str).unique()[:8])
print("GDP quarterly points:", gdp_q.shape[0])
if not gdp_q.empty:
    print("GDP range:", gdp_q.index.min().date(), "→", gdp_q.index.max().date())
display(gdp_q.head(4))

Unique 'quarter' samples: ['2012.0-Q1' '2012.0-Q2' '2012.0-Q3' '2012.0-Q4' '2013.0-Q1' '2013.0-Q2'
 '2013.0-Q3' '2013.0-Q4']
Normalized samples: ['2012-Q1' '2012-Q2' '2012-Q3' '2012-Q4' '2013-Q1' '2013-Q2' '2013-Q3'
 '2013-Q4']
GDP quarterly points: 50
GDP range: 2013-03-31 → 2025-06-30


__q
2013-03-31    4.296010
2013-06-30    6.447099
2013-09-30    7.337749
2013-12-31    6.534983
Freq: QE-DEC, Name: gdp_yoy, dtype: float64

In [7]:


def load_repo_monthly_to_quarter(path: Path):
    raw = pd.read_csv(path)

    # 1) find date & rate columns
    dcol = pick(raw, ['DATE','Date','Month','Period'])
    vcol = pick(raw, ['REPO_RATE_PERCENT','Repo','Rate','Policy Rate','REPO'])
    if dcol is None or vcol is None:
        raise ValueError("Repo CSV must have a date column (DATE/Month/Period) and a value column (Repo/Rate/REPO_RATE_PERCENT).")

    # 2) parse dates & clean numeric
    df = parse_dates_in_df(raw, dcol)
    val = (df[vcol].astype(str)
                  .str.replace(',', '', regex=False)
                  .str.replace('%', '', regex=False)
                  .str.strip())
    val = pd.to_numeric(val, errors='coerce')

    # 3) detect scaling (percent vs fraction)
    # typical RBI repo is ~4–10; if values look like 0.04–0.10, scale by 100
    median_abs = val.dropna().abs().median()
    if pd.notna(median_abs) and median_abs < 1:  # e.g., 0.065
        val = val * 100.0

    df = df.assign(val=val).dropna(subset=['val'])

    # 4) normalize to month-end stamps, forward-fill missing months
    # (so quarter-end 'last' is well-defined even if a month is missing)
    m_end = (df.set_index(dcol)['val']
               .to_period('M').to_timestamp('M')
               .sort_index())

    # create a complete monthly index, forward-fill within series
    full_m = pd.date_range(m_end.index.min(), m_end.index.max(), freq='M')
    m_end = m_end.reindex(full_m).ffill()

    # 5) quarter-end level & QoQ change in bps
    repo_q = m_end.resample('Q').last().rename('repo')                 # % at quarter-end
    repo_chg_bps = (repo_q.diff() * 100.0).rename('repo_chg_bps')      # Δ in basis points

    # 6) coverage diagnostics
    by_year_months = pd.Series(1, index=full_m).groupby(pd.Grouper(freq='Y')).size()
    print(f"[Repo] months covered: {int(by_year_months.median())} median months/yr; "
          f"quarters: {repo_q.shape[0]} ({repo_q.index.min().date()} → {repo_q.index.max().date()})")

    return repo_q, repo_chg_bps

# ---- run the loader
repo_q, repo_chg_bps = load_repo_monthly_to_quarter(RAW/'Repo_Rate_Monthly_2010_2025.csv')

# quick peek
print(repo_q.tail())
print(repo_chg_bps.tail())


[smart_parse_dates] picked: infer | parsed=188 | median months/yr=12 | unique months=12
[Repo] months covered: 12 median months/yr; quarters: 63 (2010-03-31 → 2025-09-30)
2024-09-30    6.50
2024-12-31    6.50
2025-03-31    6.25
2025-06-30    5.50
2025-09-30    5.50
Freq: QE-DEC, Name: repo, dtype: float64
2024-09-30     0.0
2024-12-31     0.0
2025-03-31   -25.0
2025-06-30   -75.0
2025-09-30     0.0
Freq: QE-DEC, Name: repo_chg_bps, dtype: float64


In [8]:
# ---- Rainfall seasonal anomaly from your monthly file (2012–2025) ----
# File columns (confirmed): year, month ("Jan".."Dec"), rainfall_mm, good_rainfall_mm, anomaly_mm

rain = pd.read_csv(RAW/'AnnualRainfall_with_Good_and_Anomaly_2012_2025.csv')

# Flexible column picks (if you already have pick(), you can use it; here we reference directly)
ycol, mcol = 'year', 'month'
obs_col, norm_col, mm_anom_col = 'rainfall_mm', 'good_rainfall_mm', 'anomaly_mm'  # we will *not* use anomaly_mm

# Normalize month text -> month number (1..12)
mmap = {m[:3].lower(): i for i, m in enumerate(
    ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], start=1)}
rain['mon'] = rain[mcol].astype(str).str[:3].str.lower().map(mmap).astype(int)

# Keep Southwest monsoon months: Jun(6)–Sep(9)
mons = rain[rain['mon'].between(6, 9)].copy()

# --- QA 1: ensure each monsoon season has its 4 months ---
miss = (mons.groupby(ycol)['mon']
            .nunique()
            .rename('monsoon_months')
            .reset_index())
bad = miss[miss['monsoon_months'] < 4]
if not bad.empty:
    print("⚠ Monsoon months missing in years:", bad[ycol].tolist())

# Aggregate to seasonal totals per year (use *observed* and *normal*; ignore anomaly_mm to avoid sign confusion)
grp = mons.groupby(ycol, as_index=False).agg(
    obs_mm  = (obs_col,  'sum'),
    norm_mm = (norm_col, 'sum'),
    # anom_mm_sum = (mm_anom_col, 'sum')  # optional: cross-check below
)

# IMD-style anomaly%: (observed - normal)/normal * 100  (positive = above normal)
grp['rain_anom_pct'] = (grp['obs_mm'] - grp['norm_mm']) / grp['norm_mm'] * 100.0

# Optional cross-check using your provided anomaly_mm (remember: good - observed)
# If you want to verify signs, uncomment:
# check = mons.groupby(ycol, as_index=False).agg(anom_mm_sum=(mm_anom_col, 'sum'),
#                                                norm_mm_sum=(norm_col, 'sum'))
# check['alt_pct_from_file'] = (-check['anom_mm_sum'] / check['norm_mm_sum']) * 100.0
# print(check[[ycol,'alt_pct_from_file']].head())

# Stamp each year's anomaly at Sep-30; forward-fill within the same year → quarterly series
rain_idx = pd.to_datetime(grp[ycol].astype(int).astype(str) + '-09-30')
rain_q = (pd.Series(grp['rain_anom_pct'].values, index=rain_idx)
            .resample('Q').ffill()
            .rename('rain_anom'))

print("Rainfall seasonal years covered:", grp[ycol].tolist())
print("Rainfall quarterly points:", rain_q.shape[0], "| Range:", rain_q.index.min().date(), "→", rain_q.index.max().date())


Rainfall seasonal years covered: [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
Rainfall quarterly points: 53 | Range: 2012-09-30 → 2025-09-30


In [9]:
# 1) Build the modeling DataFrame on the shared quarterly intersection
parts = [excess_ret, midcap_qret, nifty_qret, rain_q, cpi_yoy_q, gdp_q, repo_chg_bps]
qdf = pd.concat(parts, axis=1, join='inner')  # inner = strict intersection of all indices
qdf.index = qdf.index.to_period('Q').to_timestamp('Q')
# 2) Create lags (t-1) and the prediction target (t+1)
qdf['ret_prev_q']    = qdf['midcap_qret'].shift(1)
qdf['rain_anom_lag'] = qdf['rain_anom'].shift(1)
qdf['cpi_yoy_lag']   = qdf['cpi_yoy'].shift(1)
qdf['gdp_yoy_lag']   = qdf['gdp_yoy'].shift(1)
qdf['repo_chg_lag']  = qdf['repo_chg_bps'].shift(1)
qdf['excess_next_q'] = qdf['excess_ret'].shift(-1)

# 3) Clean rows created by shifting and print a quick QA
qdf = qdf.dropna(how="any").copy()
print("Final quarterly rows:", len(qdf), "| Range:", qdf.index.min().date(), "→", qdf.index.max().date())
qdf[['midcap_qret','nifty_qret','excess_ret','rain_anom','cpi_yoy','gdp_yoy','repo_chg_bps']].head(6)


Final quarterly rows: 44 | Range: 2014-06-30 → 2025-03-31


,midcap_qret,nifty_qret,excess_ret,rain_anom,cpi_yoy,gdp_yoy,repo_chg_bps
2014-06-30,0.288472,0.135311,0.153161,69.879102,7.859486,8.023963,0.0
2014-09-30,0.028963,0.046437,-0.017474,-7.449626,6.681568,8.704109,0.0
2014-12-31,0.102077,0.039913,0.062164,-7.449626,4.054538,5.922736,0.0
2015-03-31,0.033169,0.025149,0.008021,-7.449626,5.272440,7.112080,-50.0
2015-06-30,0.000646,-0.014427,0.015073,-7.449626,5.090809,7.592544,-25.0
2015-09-30,-0.001933,-0.050140,0.048207,-6.171560,3.948304,8.033806,-50.0


In [ ]:
qdf_out = qdf.copy()
qdf_out['excess_next_q'] = qdf_out['excess_ret'].shift(-1)
qdf_out['ret_prev_q']    = qdf_out['midcap_qret'].shift( -1 )  
qdf_final = qdf.copy().reset_index().rename(columns={'index':'date_q'})
qdf_final.to_csv(PROC/'quarterly_features.csv', index=False)
